In [1]:
#import pandas as pd
import re
from IPython.display import Markdown
from ThaiTextPrepKit.typo_patterns import patterns
from ThaiTextPrepKit import polars_pretextkit as preprocess
import polars as pl
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from bs4 import BeautifulSoup

In [114]:
thai_consonants = "กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ"
vowel_typo = '่้๊๋็ีัเ์ื?' # ่ ้ ๊ ๋ ็ ี ั เ ื ์ ?
thai_tonemarks = (
#  ็ ่ ้ ๊ ๋ ์ ํ ๎
"""
\u0e47\u0e48\u0e49\u0e4a\u0e4b\u0e4c\u0e4d\u0e4e
"""
)
thanthakhat = (
# ์ ํ ๎
"""
\u0e4c\u0e4d\u0e4e
"""
)
thai_complete_end_vowels = (
# ะ ำ ๅ ๆ
"""
\u0e30\u0e33\u0e45\u0e46
"""
)

patterns = [
    (re.compile(rf'(ๅ)', re.IGNORECASE), 'า'),
    (re.compile(rf'(แอ๊บ)|(แอ[พปฟผ]*[พปฟผ]*ลิเคช[ัี]*[่้๊๋็ีัเ]น)|(แอ[่้๊๋็ีัเ]*[พปฟผฯ][ฯ]*(?!เปิ[่้๊๋็ีัเ]*ล))|ap[p]*lication|(?<![A-Za-z])app(?![A-Za-z])', re.IGNORECASE), 'แอปพลิเคชัน'),
    (re.compile(rf'(?<![A-Za-z])apple(?![A-Za-z])|([เแ]อ[่้๊๋็ีัเ]*[ปผแบยลำพะฟห][เด้][ปบผ][ิฺอื]*[ลน])', re.IGNORECASE), 'แอปเปิ้ล'),
    (re.compile(rf'(scan|แสกน)', re.IGNORECASE), 'สแกน'),
    (re.compile(rf'(time)', re.IGNORECASE), 'เวลา'),
    (re.compile(rf'(ใข้วาน)|([ไใฝำ]*[ชข][{vowel_typo}]*[ง|ว|ฝ][า|ส|่]น)', re.IGNORECASE), 'ใช้งาน'),
    (re.compile(rf'((ใข้ว่าย)|([ไใฝำ]*[ชข][{vowel_typo}]*[งวส][{vowel_typo}]*[่าส]ย))', re.IGNORECASE), 'ใช้ง่าย'),
    (re.compile(rf'(([ไใฝำ]*[ชข][{vowel_typo}]*[นยบ][{vowel_typo}]*[่าส]ก))', re.IGNORECASE), 'ใช้ยาก'),
    (re.compile(rf'(?<!baht\s)((inter)*net)(?!\sbanking)|(อินเ[ทต]อ[ร]*[ื์]*)เ[นฯณรยญ][{vowel_typo}]*[ตจคดกทมน]|([อแิ][อิืฺ์ี]*[รนณฯญย][ดเ้][ทตมคจ][{vowel_typo}][แอิ]*[ร]*[ื์]*[รณนฯย]*[ดเ้][รณนฯย][{vowel_typo}]*[คตจทม๖?]*[คตจทม๖?{vowel_typo}]*[คตจทม๖?{vowel_typo}]*)|(เน[{vowel_typo}]*[ตท?๖][ื์]*[ตท?๖]*[ื์]*)', re.IGNORECASE), 'อินเทอร์เน็ต'),
    (re.compile(rf'(บั[นรญณ]ช[ีร])|(แอ[{thai_tonemarks}]*ค[ฯ]*(เค[{thai_tonemarks}]*า[ทตค]*[ื์]*)*)|(ac[c]*ount)|(?<=[{thai_consonants} ])(acc)(?![A-Za-z])', re.IGNORECASE), 'บัญชี'),
    (re.compile(rf'(sms)', re.IGNORECASE), 'ข้อความ'),
    (re.compile(rf'((?<!\S)a[td]m(?![A-Za-z])|ตู้[ ]*atm|เอทีเอ[{thai_tonemarks}]*ม)', re.IGNORECASE), 'เครื่องอัตโนมัติ'),
    (re.compile(rf'(พ[.]*น[.]*ง[.]*)|(พนง|พนง\.)|(พน[ั]กง[า]*น)', re.IGNORECASE), 'พนักงาน'),
    (re.compile(rf'(system|รบบ)', re.IGNORECASE), 'ระบบ'),
    (re.compile(rf'(slip|สลิ[บผพ])', re.IGNORECASE), 'สลิป'),
    (re.compile(rf'(error|เอ[{thai_tonemarks}]*อเร[{thai_tonemarks}]*อ)', re.IGNORECASE), 'ผิดพลาด'),
    (re.compile(rf'(เวิน|ฌงิน|เงิฯ|เงฺน)', re.IGNORECASE), 'เงิน'),
    (re.compile(rf'(ร[ะ]*[ฟหก][ัะีํ๊]ส)', re.IGNORECASE), 'รหัส'),
    (re.compile(rf'(\bpin(?![A-Za-z])|พิ[นณฯ](?!า))|(pwd|pa[s]*sword|pass)|(ร[ะ]*[ฟหกฆ][{vowel_typo}]*[าสว][ฟผป][{vowel_typo}][รนยฯ])|([พภ]า[ร]*[์]*[สดทต][เด้]ว[ิื]*[อ]*[ร]*[์ื]*[กดเตท])', re.IGNORECASE), 'รหัสผ่าน'),
    (re.compile(rf'(อัต[ิ]*โนมัต[ิ]*)', re.IGNORECASE), 'อัตโนมัติ'),
    (re.compile(rf'(เบอ[ร]*[์]*โท[ร]*[สศ]ั[พบ][ท]*[์]*)|(เบอ[ร์]*โท[ร]*)|(เบอ(?!ะ)[ร]*[์]*)', re.IGNORECASE), 'หมายเลขโทรศัพท์'),
    (re.compile(rf'(ไช้)', re.IGNORECASE), 'ใช้'),
    (re.compile(rf'(ไช่)', re.IGNORECASE), 'ใช่'),
    (re.compile(rf'(รุ้)', re.IGNORECASE), 'รู้'),
    (re.compile(rf'(แล[เ้่]ว)', re.IGNORECASE), 'แล้ว'),
    (re.compile(rf'(บ[.]*ช[.])|(บั[น|ร|ณ|ย]ชี)', re.IGNORECASE), 'บัญชี'),
    (re.compile(rf'(เข้ส)', re.IGNORECASE), 'เข้า'),
    (re.compile(rf'(ธุระกรมม|ธุระกรม|ธุรกรม|ธุรกรมม|ธุระกรรม|ทุระกรรม|ทุรกรรม|ทุรกรม|ทุรกรมม|ธุกรรม|ทุกรรม)', re.IGNORECASE), 'ธุรกรรม'),
    (re.compile(rf'(อัพ)', re.IGNORECASE), 'อัป'),
    (re.compile(rf'(ให่|ไห้|ไห่)', re.IGNORECASE), 'ให้'),
    (re.compile(rf'(ทันไจ|ทันจัย)', re.IGNORECASE), 'ทันใจ'),
    (re.compile(rf'(ปั[{vowel_typo}]*[ญยนณรสบฯ]หา)', re.IGNORECASE), 'ปัญหา'),
    (re.compile(rf'(อัพเดท|อัพเดต|อัปเดท|อัปเกรด|update|upgrade)', re.IGNORECASE), 'อัปเดต'),
    (re.compile(rf'(สะดวด|สดวก|สดวด|สกวก|สะกวก|สพกวก|สพดวก|convenient|convenience)', re.IGNORECASE), 'สะดวก'),
    (re.compile(rf'(login|log-in|ล็อคอิน|ล็อกอิน|ลอกอิน|ล้อกอิน|ลอคอิน|ล้อคอิน)', re.IGNORECASE), 'เข้าใช้งาน'),
    (re.compile(rf'(ลวดเร็ว|ลวดเล็ว|รวดเล็ว|ดรดเร็ว|รวดเรว|รวดดร็ว|รวดเร้ว|fast|พรวดเร็ว)|เ[ลร][{vowel_typo}]*[กดเ]*[กดเ][ลร][{vowel_typo}]*[วด]|([พ]*[พรล]ว[เด้]*[แเด้][รล][{vowel_typo}]*[วด])', re.IGNORECASE), 'รวดเร็ว'),
    (re.compile(rf'(เร้ว|ดร็ว|ดรว|เรว)', re.IGNORECASE), 'เร็ว'),
    (re.compile(rf'(อย่างง(?!ง))', re.IGNORECASE), 'อย่าง'),
    (re.compile(rf'(งง+)', re.IGNORECASE), 'งง'),
    (re.compile(rf'(บริ[กด][าส][า]*[นรฯยญณ])', re.IGNORECASE), 'บริการ'),
    (re.compile(rf'(เหตการ|เหตการณ์)', re.IGNORECASE), 'เหตุการณ์'),
    (re.compile(rf'(มาก+)', re.IGNORECASE), 'มาก'),
    (re.compile(rf'เก[ณ]([ฑพทฐ][{thanthakhat}])*|เก[รญนฯ]*([ฑพทฐ][{thanthakhat}])', re.IGNORECASE), 'เกณฑ์'),
    (re.compile(rf'(cal[l]*[ ]*center)|(คอ[นล]เซ[{thai_tonemarks}]*[นลยบญรฯ]เต[{thai_tonemarks}]*อ(ร[{thai_tonemarks}]*)*)', re.IGNORECASE), 'คอลเซ็นเตอร์'),
    (re.compile(rf'([ๆไใ]ม[{vowel_typo}]*[ๆไใำ]ด[{vowel_typo}]*)|(มั[{vowel_typo}]*ย[ๆไใำ]ด[{vowel_typo}]*)|(มั[{vowel_typo}]*ยดั[{vowel_typo}]*ย)|(มั[{vowel_typo}]*ยด[{vowel_typo}]*าย)|(มั[{vowel_typo}]*ยดร[{vowel_typo}]*[า]*ย)|([ๆไใ]ม[{vowel_typo}]*ด[{vowel_typo}]*าย)|(บ่(ด[{vowel_typo}]*าย|ดร[{vowel_typo}]*[า]*ย|[ๆไใำ]ด[{vowel_typo}]*))', re.IGNORECASE), 'ไม่ได้'),
    (re.compile(rf'(ล[{thai_tonemarks}]าช[{thai_tonemarks}]*[า])', re.IGNORECASE), 'ล่าช้า'),
    (re.compile(rf'([โดเก]ค[ห]*[วสงย][ิื]*[{vowel_typo}]*[ดคตท]*[- ]*19)|(covid[ ]*19)', re.IGNORECASE), 'covid-19'),
    (re.compile(rf'([ตคจ][{vowel_typo}]*[อิแ]*[{vowel_typo}]*[วง][กดห][่าส][รนี])', re.IGNORECASE), 'ต้องการ'),
    (re.compile(rf'จ[.]*น[.]*ท[.]', re.IGNORECASE), 'เจ้าหน้าที่'),
    (re.compile(rf'[ส]*[ะ]*เ(ส[ี]*)*ถ[ี]*ย[รนยณญ]', re.IGNORECASE), 'เสถียร'),
    (re.compile(rf'((กรุงไท[ย]*)เน[{thai_tonemarks}]*[กหดป])', re.IGNORECASE), 'กรุงไทยเน็กซ์'),
    #(re.compile(rf'(แบ[{thai_tonemarks}]*ง)[กค][์]', re.IGNORECASE), 'ธนาคาร'),
    (re.compile(rf'(แบ[{thai_tonemarks}]*ง)[กค][์]', re.IGNORECASE), 'แบงก์'),

    # New in 1.1d
    # Only support with 'ลาย___' only
    (re.compile(rf'(ล[าสษ]ย)เ[ซวง][{thai_tonemarks}]*[นฯณญยร]([ตค][{thanthakhat}])*', re.IGNORECASE), 'ลายเซ็น'),
    (re.compile(rf'คั[สดศษซตจช]([โ]*([ทตค][อ]*[ม]*))*เมอ[รสต{thanthakhat}]*เซอ[รสต{thanthakhat}]*[ห]*วิ[สวศซษตดช]', re.IGNORECASE), 'Customer Service'),
    (re.compile(rf'ซั[บปฟพฑ]พ(อ(ร[{thanthakhat}]*)*)*[ตดส]', re.IGNORECASE), 'ซัพพอร์ต'),
    (re.compile(rf'ค่า[ธะทพฑ]*ร[ร]*มเนียม', re.IGNORECASE), 'ค่าธรรมเนียม'),
    (re.compile(rf'(ผ[.]*จ[.]*ก[.]*)', re.IGNORECASE), 'ผู้จัดการ'),
    (re.compile(rf'(ส[.]*น[.]*ง[.])', re.IGNORECASE), 'สำนักงาน'),
    (re.compile(rf'ดอก[ดเแ]*บี้ย', re.IGNORECASE), 'ดอกเบี้ย'),
    (re.compile(rf'(dow[n]*)*load|(?<!อั[พปบ])(ดาว)*(น[{thanthakhat}]*)*[โด][ห]*ลด', re.IGNORECASE), 'ดาวน์โหลด'),
    (re.compile(rf'upload|อั[{thai_tonemarks}]*[พปบฟ][โด][ห]*ลด', re.IGNORECASE), 'อัปโหลด'),
    (re.compile(rf'counter|เค[{thai_tonemarks}]*า([นทต][{thanthakhat}]*)*เ[ตท]อร์', re.IGNORECASE), 'เคาน์เตอร์'),
    (re.compile(rf'อัต[ร]*[า]|อัตร[า]*', re.IGNORECASE), 'อัตรา'),

    # Base end
    #(re.compile(rf'()', re.IGNORECASE), ''),
    (re.compile(rf'(&nbsp;)', re.IGNORECASE), ''),
    (re.compile(rf'(?<=\S)\.(?=\s|$)', re.IGNORECASE), ''), # remove the full stop mark at the end of a sentence
]

In [15]:
'''def highlight_patterns(patterns, text, highlight_color="#FFFF00"):
    """
    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.

    :param patterns: A list of tuples containing regex patterns and their replacements.
    :param text: The text to search within.
    :param highlight_color: The background color to use for highlighting (default is yellow).
    :return: The text with highlighted matches.
    """
    def add_highlight(match):
        return f'<span style="background-color: {highlight_color};">{match.group(0)}</span>'
    
    highlighted_text = text
    for pattern, replacement in patterns:
        highlighted_text = re.sub(pattern, add_highlight, highlighted_text)
    
    return highlighted_text'''

'def highlight_patterns(patterns, text, highlight_color="#FFFF00"):\n    """\n    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.\n\n    :param patterns: A list of tuples containing regex patterns and their replacements.\n    :param text: The text to search within.\n    :param highlight_color: The background color to use for highlighting (default is yellow).\n    :return: The text with highlighted matches.\n    """\n    def add_highlight(match):\n        return f\'<span style="background-color: {highlight_color};">{match.group(0)}</span>\'\n    \n    highlighted_text = text\n    for pattern, replacement in patterns:\n        highlighted_text = re.sub(pattern, add_highlight, highlighted_text)\n    \n    return highlighted_text'

In [201]:
def highlight_patterns(patterns, text, highlight_color_replace="#FFFF00", highlight_color_match="#00FFFF"):
    """
    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.

    :param patterns: A list of tuples containing regex patterns and their replacements.
    :param text: The text to search within.
    :param highlight_color_replace: The background color to use for replacements (default is yellow).
    :param highlight_color_match: The background color to use for matches without replacement (default is blue).
    :return: The text with highlighted matches.
    """
    def add_highlight(match, color):
        return f'<span style="background-color: {color};">{match.group(0)}</span>'
    
    highlighted_text = text
    for pattern, replacement in patterns:
        matches = list(re.finditer(pattern, text))
        if matches:
            for match in matches:
                if replacement in match.group(0):
                    highlighted_text = highlighted_text.replace(match.group(0), add_highlight(match, highlight_color_match))
                else:
                    highlighted_text = highlighted_text.replace(match.group(0), add_highlight(match, highlight_color_replace))
    
    return highlighted_text

In [202]:
text = "I love using แอ๊บ and แอปพลิเคชัน application on my apple device."

highlighted_text = highlight_patterns(patterns, text)

In [203]:
display(Markdown(highlighted_text))

I love using <span style="background-color: #FFFF00;">แอ๊บ</span> and <span style="background-color: #00FFFF;">แอปพลิเคชัน</span> <span style="background-color: #FFFF00;">application</span> on my <span style="background-color: #FFFF00;">apple</span> device<span style="background-color: #00FFFF;">.</span>

In [143]:
highlighted_text

'I love using <span style="background-color: #FFFF00;">แอ๊บ</span> and <span style="background-color: #00FFFF;">แอปพลิเคชัน</span> <span style="background-color: #FFFF00;">application</span> on my <span style="background-color: #FFFF00;">apple</span> device<span style="background-color: #00FFFF;">.</span>'

In [184]:
def get_highlight_texts(texts: list) -> None:
    html_text = []
    for text in texts:
        html_text.append(highlight_patterns(patterns, text))
    return html_text

In [23]:
df = pl.read_excel('/Users/pa/Downloads/Test ML - Corp V2.xlsx',
                   engine='calamine')

texts = df.get_column('Reason')

In [204]:
texts = [
    'แอปพลิเคชันดี',
    'แอพลิเคชั่นดี',
    'internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ',
    'internet banking',
    'net banking',
    'อินเตอร์เน็ต',
    'อินเตอร์เน็ตแบงค์กิ้ง',
    'internet แบงกิ้ง',
    'baht net',
    'bath net',
    'net',
    'I love using แอ๊บ and แอปพลิเคชัน application on my apple device.',
]
html_texts = get_highlight_texts(texts)

for text in html_texts:
    display(Markdown(text))

<span style="background-color: #00FFFF;">แอปพลิเคชัน</span>ดี

<span style="background-color: #FFFF00;">แอพลิเคชั่น</span>ดี

internet banking <span style="background-color: #00FFFF;">ใช้งาน</span>ง่าย มีคู่มือให้ดูได้จากหน้าเว็บ

internet banking

net banking

...

<span style="background-color: #FFFF00;">internet</span> แบงกิ้ง

baht net

bath <span style="background-color: #FFFF00;">net</span>

<span style="background-color: #FFFF00;">net</span>

I love using <span style="background-color: #FFFF00;">แอ๊บ</span> and <span style="background-color: #00FFFF;">แอปพลิเคชัน</span> <span style="background-color: #FFFF00;">application</span> on my <span style="background-color: #FFFF00;">apple</span> device<span style="background-color: #00FFFF;">.</span>

In [216]:
series = pl.Series(texts)
pre_series = preprocess.preprocess_text_batches(series=series,
                                   custom_dict=None,
                                   keep_stopwords=True,
                                   keep_format=True,
                                   return_token_list=False,
                                   lower_case=False,
                                   include_pattern='/')

for t in pre_series:
    print(t)

แอปพลิเคชันดี
แอปพลิเคชันดี
internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ
internet banking
net banking
อินเทอร์เน็ต
อินเทอร์เน็ตแบงก์กิ้ง
อินเทอร์เน็ต แบงกิ้ง
baht net
bath อินเทอร์เน็ต
อินเทอร์เน็ต
I love using แอปพลิเคชัน and แอปพลิเคชัน แอปพลิเคชัน on my แอปเปิ้ล device


In [219]:
from IPython.display import HTML, display, FileLink

def generate_html_table(original_texts, preprocess_texts):
    """
    Generate an HTML table with 'original_text' and 'preprocess_text' columns.

    Args:
    - original_texts (list): List of original texts.
    - preprocess_texts (list): List of preprocessed texts.

    Returns:
    - html_content (str): String containing the HTML table.
    """
    html_content = "<table border='1'>\n"
    html_content += "<tr><th>original_text</th><th>preprocess_text</th></tr>\n"

    for original, preprocess in zip(original_texts, preprocess_texts):
        html_content += f"<tr><td>{original}</td><td>{preprocess}</td></tr>\n"

    html_content += "</table>"
    
    return html_content


In [220]:
# Example lists (replace with your data)
'''original_texts = ['<span style="background-color: #00FFFF;">แอปพลิเคชัน</span>ดี',
                  '<span style="background-color: #FFFF00;">แอพลิเคชั่น</span>ดี',
                  'internet banking <span style="background-color: #00FFFF;">ใช้งาน</span>ง่าย มีคู่มือให้ดูได้จากหน้าเว็บ',]

preprocess_texts = ['แอปพลิเคชันดี',
                    'แอปพลิเคชันดี',
                    'internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ']'''

# Generate HTML table
html_table = generate_html_table(original_texts=html_texts, preprocess_texts=pre_series.to_list())

# Display HTML in Jupyter Notebook
from IPython.display import HTML, display
display(HTML(html_table))

original_text,preprocess_text
แอปพลิเคชันดี,แอปพลิเคชันดี
แอพลิเคชั่นดี,แอปพลิเคชันดี
internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ,internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ
internet banking,internet banking
net banking,net banking
อินเตอร์เน็ต,อินเทอร์เน็ต
อินเตอร์เน็ตแบงค์กิ้ง,อินเทอร์เน็ตแบงก์กิ้ง
internet แบงกิ้ง,อินเทอร์เน็ต แบงกิ้ง
baht net,baht net
bath net,bath อินเทอร์เน็ต


In [221]:
# Create a downloadable link for the HTML content
with open('table.html', 'w') as f:
    f.write(html_table)

FileLink('table.html')

/Users/pa/Documents/GitHub Repository Clone/DataPrepKit/table.html